In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from tensorflow import keras
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.10.0
sys.version_info(major=3, minor=9, micro=13, releaselevel='final', serial=0)
matplotlib 3.5.2
numpy 1.23.1
pandas 1.4.4
sklearn 1.1.2
tensorflow 2.10.0
keras.api._v2.keras 2.10.0


In [2]:
if os.environ['COMPUTERNAME'] == 'DESKTOP-EQAO3M5':
  computer_flag = 'home'
else:
  computer_flag = 'office'

if computer_flag == 'home':
  file_dir = "F:/github/pythonScript/seafog/data/collection/"
else:
  file_dir = "H:/github/python/seafog/data/collection/"

In [3]:
def linear_vis(x):
    if(x <= 1000.0):
      y = x/1000.0
    elif(x > 1000.0 and x <= 10000.0):
      y = (x-1000.0)/9000.0 + 1.0
    elif( x > 10000.0  and x <= 30000.0):
      y = (x-10000.0)/20000 + 2.0
    else:
      y = 3.0
    return y

def reverse_linear_vis(x):
    if(x<0):
      y = 1
    elif(x <= 1.0):
      y = x*1000.0
    elif(x <= 2.0):
      y = (x - 1.0)*9000.0 + 1000.0
    elif(x <= 3.0):
      y = (x-2.0)*20000 + 10000.0
    else:
      y = 30000.0
    return y

def setDataset(df, x_columns):
    df['t_td'] =  df['t2mm'] - df['t2md']
    df['td_sst'] =  df['t2md'] - df['sstk']
    df['t_sst'] =  df['t2mm'] - df['sstk']
    df['delta_theta'] = df['theta925'] - df['theta1000']
    df['delta_theta_e'] = df['theta_e925'] - df['theta_e1000']
    df_x = df.loc[:, x_columns]
    df_y = df['station_vis_linear']
    _x = df_x.to_numpy()
    _y = df_y.to_numpy()
    return (_x, _y)

def getScore(x,y, model):
    predictions = model(x).numpy()
    y_p = predictions[:,0]
    df_s = pd.DataFrame({'prediction':y_p, 'y':y})
    df_s_fog = df_s.loc[df_s['y']<=1.0]
    df_s_mist = df_s.loc[(df_s['y']>1.0) & (df_s['y']<=2.0)]
    df_s_clear = df_s.loc[df_s['y']>2.0]
    print('实况为雾时:', end=' ')
    getLevelRatio(df_s_fog)
    # print('-------------')
    print('实况为轻雾时:', end=' ')
    # print('-------------')
    getLevelRatio(df_s_mist)
    print('实况为无雾时:', end=' ')
    getLevelRatio(df_s_clear)

def getLevelRatio(df):
    length = len(df)
    ratio_fog = len(df.loc[df['prediction']<=1.0])/length
    ratio_mist = len(df.loc[(df['prediction']>1.0) & (df['prediction']<=2.0)])/length
    ratio_clear = len(df.loc[df['prediction']>2.0])/length
    print(f'比例: 雾:{ratio_fog}, 轻雾{ratio_mist}, 无雾{ratio_clear}')
    return (ratio_fog, ratio_mist, ratio_clear)

In [4]:
fog_dataset_hdf = os.path.normpath(os.path.join(file_dir, './fog_dataset_hdf66_fc120h.h5'))
store_dataset = pd.HDFStore(fog_dataset_hdf, mode='r')
print(store_dataset.keys())
df_train = store_dataset.get('train')
df_valid =store_dataset.get('valid')
df_test = store_dataset.get('test')
df_valid_test = store_dataset.get('valid_test')
x_columns = ['t_td', 'td_sst','t_sst','v100', 'v10m', 'u100', 'u10m', 't2mm', 't2md', 'sstk','year_sin','year_cos', 'day_sin', 'day_cos','delta_theta','delta_theta_e','theta_e925']

(train_x, train_y) = setDataset(df_train, x_columns)
(valid_x, valid_y) = setDataset(df_valid, x_columns)
(test_x,  test_y)  = setDataset(df_test, x_columns)
(valid_test_x, valid_test_y) = setDataset(df_valid_test, x_columns)



print(train_x.shape)
print(train_y.shape)



['/test', '/train', '/valid', '/valid_test']
(499005, 17)
(499005,)


In [5]:
scaler = StandardScaler()
train_x_scaled_fit = scaler.fit(train_x)
params_fit = train_x_scaled_fit.get_params()
print(params_fit)

{'copy': True, 'with_mean': True, 'with_std': True}


In [10]:
print(train_x_scaled_fit.scale_)
print(train_x_scaled_fit.mean_)
print(train_x_scaled_fit.var_)

[ 1.98330058  4.09737296  3.26146912  4.18864711  3.26081599  3.38025032
  2.71565316  3.97426555  4.02316612  4.922445    0.67417551  0.62579616
  0.66347492  0.71449761  1.74456929  4.96332235 13.30171746]
[ 2.5973422  -1.90860388  0.68873832 -0.2048123  -0.53910242 -4.53252654
 -3.56074319 22.90593504 20.30859284 22.21719672  0.08056004  0.38389665
 -0.14603633  0.16723512  3.32154619 -2.99002722 62.7480493 ]
[  3.93348118  16.78846518  10.63718083  17.5447646   10.6329209
  11.4260922    7.37477208  15.79478667  16.18586566  24.2304648
   0.45451261   0.39162083   0.44019898   0.51050683   3.04352201
  24.63456871 176.93568747]


In [13]:
train_x.var(axis=0)

array([  3.93348118,  16.78846518,  10.63718083,  17.5447646 ,
        10.6329209 ,  11.4260922 ,   7.37477208,  15.79478667,
        16.18586566,  24.2304648 ,   0.45451261,   0.39162083,
         0.44019898,   0.51050683,   3.04352201,  24.63456871,
       176.93568747])

In [ ]:
# (train-mean)/std

# mean=train_data.mean(axis=0)
# train_data-=mean
# std=train_data.std(axis=0)
# train_data/=std
# test_data-=mean
# test_data/=std

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()
